In [ ]:
import sys
sys.path.append('../')

In [ ]:
import glob
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
EXPERIMENT_DIR = "../experiments_iterative"

COMPUTE_LIKELIHOODS = False

In [ ]:
DOMAIN = "logo"

CONDITION_STITCH = "stitch"
CONDITION_STITCH_CODEX = "stitch_codex"
CONDITION_ORACLE = "oracle"

In [ ]:
NUMERIC_SORT = lambda x: (len(x), x)

In [ ]:
def get_log_likelihoods(condition, split):
    results_pattern = os.path.join(EXPERIMENT_DIR, "outputs", "domains", DOMAIN, condition, f"seed_*")
    
    seed_paths = sorted(glob.glob(results_pattern), key=NUMERIC_SORT)    
    if not seed_paths:
        raise ValueError(results_pattern)
    
    df_list = []
    for seed_path in seed_paths:
        random_seed = int(seed_path[len(results_pattern)-1:])
        print(f"Condition: {condition}, split: {split}, seed: {random_seed}")
        
        df = get_log_likelihoods_single_replication(condition, split, random_seed)
        df["random_seed"] = random_seed
        df_list.append(df)
        
    df_concat = pd.concat(df_list, axis=0).reset_index(drop=True)
    
    # Compute the mean over replications
    AGG_COLS = ["n_frontiers", "description_length"]
    if COMPUTE_LIKELIHOODS:
        AGG_COLS += ["log_likelihood"]
    df_out = df_concat.groupby(["batch_size", "task_name"], sort=False, as_index=False)[AGG_COLS].mean()
    
    return df_out
    

def get_log_likelihoods_single_replication(condition, split, random_seed):
    data = []
    
    results_pattern = os.path.join(EXPERIMENT_DIR, "outputs", "domains", DOMAIN, condition, f"seed_{random_seed}", f"{condition}_*")

    for path in sorted(glob.glob(results_pattern), key=NUMERIC_SORT):
        config_json_path = os.path.join(path, "config.json")
        with open(config_json_path, "r") as f:
            config = json.load(f)
        global_batch_size = config["experiment_iterator"]["task_batcher"]["params"][
            "global_batch_size"
        ]

        test_likelihoods_json_path = os.path.join(path, "0", "test_likelihoods.json")
        with open(test_likelihoods_json_path, "r") as f:
            likelihoods_data = json.load(f)

        stitch_frontiers_json_path = os.path.join(
            path, "0", split, "stitch_frontiers.json"
        )
        with open(stitch_frontiers_json_path, "r") as f:
            stitch_frontiers_data = json.load(f)

        for task_name, dl_list in likelihoods_data["description_lengths_by_task"][
            "test"
        ].items():
            d = {
                    "batch_size": global_batch_size,
                    "task_name": task_name,
                    "description_length": dl_list[0],
                    "n_frontiers": len(stitch_frontiers_data["frontiers"]),
                }
            if COMPUTE_LIKELIHOODS:
                d["log_likelihood"] = likelihoods_data["log_likelihoods_by_task"]["test"][task_name][0]
                
            data.append(d)

    df = pd.DataFrame(data)
    return df

In [ ]:
df1 = get_log_likelihoods(CONDITION_STITCH, "train")
df1["condition"] = "Stitch"

df2 = get_log_likelihoods(CONDITION_STITCH_CODEX, "train")
df2["condition"] = "Stitch + Codex"

df3 = get_log_likelihoods(CONDITION_ORACLE, "test")
df3["condition"] = "Oracle"

In [ ]:
df = pd.concat([df1, df2, df3], axis=0).reset_index(drop=True)
df.batch_size = df.batch_size.astype(int)
BATCH_SIZES = sorted(df.batch_size.unique().tolist())
BATCH_SIZES

In [ ]:
df

In [ ]:
sns.barplot(data=df, x="batch_size", y="n_frontiers", hue="condition")

In [ ]:
sns.pointplot(data=df, x="batch_size", y="description_length", hue="condition")

In [ ]:
g = sns.lineplot(data=df, x="batch_size", y="description_length", hue="condition")
g.set(xscale="log");

In [ ]:
if COMPUTE_LIKELIHOODS:
    sns.pointplot(data=df, x="batch_size", y="log_likelihood", hue="condition")

In [ ]:
if COMPUTE_LIKELIHOODS:
    g = sns.lineplot(data=df, x="batch_size", y="log_likelihood", hue="condition")
    g.set(xscale="log");

In [ ]:
if COMPUTE_LIKELIHOODS:
    sns.scatterplot(data=df, x="description_length", y="log_likelihood", hue="condition", alpha=0.25)

# What programs does Codex generate?

In [ ]:
def get_codex_programs():
    df_list = []
    for batch_size in BATCH_SIZES:
        path = os.path.join(EXPERIMENT_DIR, "outputs", "domains", DOMAIN, CONDITION_STITCH_CODEX, f"{CONDITION_STITCH_CODEX}_{batch_size}/0/codex_query_results.json")
        with open(path, "r") as f:
            codex_query_results = json.load(f)

        data = []
        for p in codex_query_results["prompt_programs"]:
            data.append(
                {
                    "program": p,
                    "origin": "train",
                    "valid": True,
                }
            )
        for p in codex_query_results["programs_valid"]:
            data.append(
                {
                    "program": p,
                    "origin": "codex",
                    "valid": True,
                }
            )
        for p in codex_query_results["programs_invalid"]:
            data.append(
                {
                    "program": p,
                    "origin": "codex",
                    "valid": False,
                }
            )

        df = pd.DataFrame(data)
        df["program_str_len"] = df.program.str.len()
        df["batch_size"] = batch_size

        train_programs = set(df[df["origin"] == "train"]["program"])
        df["copied_from_train"] = [
            (row["origin"] == "codex") and (row["program"] in train_programs)
            for _, row in df.iterrows()
        ]

        df_list.append(df)

    #         print(
    #             f"+ From {batch_size} training programs, Codex generated {len(df[df.origin == 'codex'])} programs, for a total of {len(df)} programs."
    #         )
    #         print(
    #             f"+ Of these, {len(df[(df.origin == 'codex') & (df.valid)])} were valid and {len(df[(df.origin == 'codex') & (~df.valid)])} were invalid."
    #         )
    #         print(
    #             f"+ In total, there were {df['program'].nunique()} unique programs; {df[df.valid]['program'].nunique()} were valid."
    #         )

    #         copied_programs = set(codex_query_results["programs_valid"]) & set(
    #             codex_query_results["prompt_programs"]
    #         )
    #         print(
    #             f"+ {len(copied_programs)} of the Codex programs were direct copies from the training data."
    #         )

    return pd.concat(df_list).reset_index(drop=True)

In [ ]:
df_codex = get_codex_programs()

In [ ]:
df_codex

In [ ]:
plt.title("Percentage of valid programs")
sns.barplot(data=df_codex, x="batch_size", y="valid", hue="origin");

In [ ]:
plt.title("Program string length")
sns.violinplot(data=df_codex, x="batch_size", y="program_str_len", hue="origin");

In [ ]:
plt.title("Count of unique programs")
sns.pointplot(
    data=df_codex.groupby("batch_size").nunique().reset_index(),
    x="batch_size",
    y="program",
);

In [ ]:
plt.title("Count of programs copied from train")
sns.pointplot(
    data=df_codex.groupby("batch_size").sum().reset_index(),
    x="batch_size",
    y="copied_from_train",
);

In [ ]:
sns.displot(
    data=df_codex, x="program_str_len", hue="valid", col="origin", row="batch_size"
);

# What inventions are in the libraries?

In [ ]:
def get_library_inventions(condition, split):
    data = []
    for batch_size in BATCH_SIZES:
        path = os.path.join(EXPERIMENT_DIR, "outputs", "domains", DOMAIN, condition, f"{condition}_{batch_size}", "0", split, "stitch_output.json")
        # path = f"../experiments/outputs/{DOMAIN}/{experiment_id}/{experiment_id}_{batch_size}/0/{split}/stitch_output.json"

        with open(path, "r") as f:
            stitch_output_data = json.load(f)

        df = pd.DataFrame(stitch_output_data["invs"])[
            ["name", "arity", "utility", "multiplier", "body", "dreamcoder"]
        ]
        df["batch_size"] = batch_size
        data.append(df)

    return pd.concat(data, axis=0).reset_index(drop=True)

In [ ]:
df_libraries_baseline = get_library_inventions(CONDITION_STITCH, "train")
df_libraries_baseline["condition"] = "baseline"

df_libraries_codex = get_library_inventions(CONDITION_STITCH_CODEX, "train")
df_libraries_codex["condition"] = "codex"

df_libraries_test = get_library_inventions(CONDITION_ORACLE, "test")
df_libraries_test["condition"] = "test"

df_libraries = pd.concat(
    [df_libraries_baseline, df_libraries_codex, df_libraries_test], axis=0
).reset_index(drop=True)

In [ ]:
data_overlap = []
for batch_size, group in df_libraries.groupby("batch_size"):
    fns_baseline = set(group[group.condition == "baseline"].dreamcoder)
    fns_codex = set(group[group.condition == "codex"].dreamcoder)
    fns_test = set(group[group.condition == "test"].dreamcoder)
    data_overlap.append(
        {
            "batch_size": int(batch_size),
            "utility_baseline": group[group.condition == "baseline"]["utility"].sum(),
            "utility_codex": group[group.condition == "codex"]["utility"].sum(),
            "utility_test": group[group.condition == "test"]["utility"].sum(),
            "n_baseline": len(fns_baseline),
            "fns_baseline": fns_baseline,
            "n_codex": len(fns_codex),
            "fns_codex": fns_codex,
            "n_test": len(fns_test),
            "fns_test": fns_test,
            "n_baseline_only": len(fns_baseline - fns_codex),
            "fns_baseline_only": fns_baseline - fns_codex,
            "n_codex_only": len(fns_codex - fns_baseline),
            "fns_codex_only": fns_codex - fns_baseline,
            "n_overlap": len(fns_baseline & fns_codex),
            "fns_overlap": (fns_baseline & fns_codex),
            "n_baseline_test": len(fns_baseline & fns_test),
            "n_codex_test": len(fns_codex & fns_test),
        }
    )

In [ ]:
df_overlap = pd.DataFrame(data_overlap)
df_overlap

In [ ]:
df_overlap_long = df_overlap.rename(dict(zip(["n_baseline_only", "n_codex_only", "n_overlap"], ["baseline_only", "codex_only", "overlap"])), axis="columns")
df_overlap_long = df_overlap_long.melt(id_vars="batch_size", value_vars=["baseline_only", "codex_only", "overlap"], var_name="library", value_name="Inventions (count)")

plt.title("Overlap between Baseline and Codex inventions")
sns.barplot(data=df_overlap_long, x="batch_size", y="Inventions (count)", hue="library")

In [ ]:
df_overlap_long = df_overlap.rename(dict(zip(["n_baseline_test", "n_codex_test", "n_test"], ["Stitch", "Stitch + Codex", "Oracle"])), axis="columns")
df_overlap_long = df_overlap_long.melt(id_vars="batch_size", value_vars=["Stitch", "Stitch + Codex", "Oracle"], var_name="library", value_name="Oracle inventions discovered (count)")

plt.title("Discovery of oracle (test set) inventions")
sns.pointplot(data=df_overlap_long, x="batch_size", y="Oracle inventions discovered (count)", hue="library")

In [ ]:
df_overlap_long

In [ ]:
from matplotlib_venn import venn2, venn3

In [ ]:
os.makedirs("library_venn_diagrams", exist_ok=True)
os.makedirs("library_venn_diagrams/venn2", exist_ok=True)
os.makedirs("library_venn_diagrams/venn3", exist_ok=True)

In [ ]:
for i, row in df_overlap.iterrows():
    plt.figure()
    venn2(
        subsets=(row["n_baseline_only"], row["n_codex_only"], row["n_overlap"]),
        set_labels=("Baseline", "Codex"),
    )
    plt.title("Batch size: " + str(row["batch_size"]))
    plt.savefig(f"library_venn_diagrams/venn2/batch_{row['batch_size']:03d}.png", dpi=144)

In [ ]:
for batch_size, group in df_libraries.groupby("batch_size"):
    fns_baseline = set(group[group.condition == "baseline"].dreamcoder)
    fns_codex = set(group[group.condition == "codex"].dreamcoder)
    fns_test = set(group[group.condition == "test"].dreamcoder)
    plt.figure()
    venn3(
        [fns_baseline, fns_codex, fns_test],
        set_labels=("Baseline", "Codex", "Oracle"),
    )
    plt.title("Batch size: " + str(batch_size))
    plt.savefig(f"library_venn_diagrams/venn3/batch_{int(batch_size):03d}.png", dpi=144)